In [12]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from pandas_profiling import ProfileReport

from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor


In [2]:

def get_data(img_path):
    img = plt.imread(img_path)
    #img -> 150x150 input image
    output_pixel = img[45][75][0] # last index is to be corrected
    neighbours = [
        img[50][55][0],
        img[50][65][0],
        img[50][75][0],
        img[50][85][0],
        img[50][95][0]
    ]
    cropped_img = img[50:100, 50:100]
    return neighbours, output_pixel


In [3]:

def get_df(dir):
    df = pd.DataFrame(columns=['input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'output'])
    for i in os.listdir(dir):
        row = get_data(dir+i)
        df.loc[len(df.index)] = row[0] +[row[1]]
    return df


In [9]:

df = get_df('/home/anirudh/Desktop/images/')
print(df)

      input_1     input_2     input_3     input_4     input_5      output
0    0.313726    0.721569    0.580392    0.168627    0.180392    0.580392
1   47.000000   47.000000   47.000000   47.000000   47.000000   47.000000
2    0.925490    0.925490    0.925490    0.925490    0.925490    0.925490
3   53.000000   53.000000   51.000000   51.000000   50.000000   51.000000
4    0.266667    0.266667    0.266667    0.266667    0.266667    0.266667
5    0.078431    0.078431    0.078431    0.078431    0.078431    0.078431
6  233.000000  234.000000  228.000000  224.000000  226.000000  226.000000
7    0.000000    0.000000    0.000000    0.000000    0.000000    0.000000


In [10]:
profile = ProfileReport(df, title="Pandas Profiling Report")

In [11]:
profile.to_file("basic_report.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  4.43it/s]


In [ ]:
X = df[df.columns[:5]]
y = df['output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
regr.score(X_test, y_test)

In [ ]:
neigh = KNeighborsRegressor(n_neighbors=2)
neigh.fit(X, y)
neigh.score(X_test, y_test)